# Importation des librairies

In [ ]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.decomposition import PCA

import importlib
spec = importlib.util.spec_from_file_location("preprocessing", "..\\utils\\preprocessing.py")
preprocessing = importlib.util.module_from_spec(spec)
spec.loader.exec_module(preprocessing)

spec = importlib.util.spec_from_file_location("fspliter", "..\\utils\\files_spliter.py")
fspliter = importlib.util.module_from_spec(spec)
spec.loader.exec_module(fspliter)

spec = importlib.util.spec_from_file_location("results", "..\\utils\\results.py")
results = importlib.util.module_from_spec(spec)
spec.loader.exec_module(results)

# Lecture des données et preprocessing

In [ ]:
mouse1 = fspliter.get_mice(0)
mouse1 = fspliter.retrieve_day(mouse1, 3)
mouse1 = preprocessing.do_preprocessing(mouse1, 'RN')

mouse2 = fspliter.get_mice(1)
mouse2 = fspliter.retrieve_day(mouse2, 3)
mouse2 = preprocessing.do_preprocessing(mouse2, 'RN')

# Séparation en sets d'entrainement et de test

In [ ]:
pca = PCA(n_components=50)
train = mouse1
test = mouse2

#remove filename columns
train = train.drop(['filename'], axis=1)
test = test.drop(['filename'], axis=1)

#state column as label and the others as features
train_features = train.drop(['state'], axis=1)
train_labels = train['state']
pca.fit(train_features)
train_features = pca.transform(train_features)

test_features = test.drop(['state'], axis=1)
test_labels = test['state']
test_features = pca.transform(test_features)
print(train_features.shape)


# Model

In [ ]:
clf = svm.SVC()
clf.fit(train_features, train_labels)
predict = clf.predict(test_features)

# Résultats

In [ ]:
results.scores(test_labels, predict, mouse2.state.unique())